In [11]:
import torch
import torchvision
from torchvision import models, transforms
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import json

In [2]:
print("Torch Version: ", torch.__version__)
print("Torchvision Version: ", torchvision.__version__)

Torch Version:  2.3.1
Torchvision Version:  0.18.1


In [3]:
use_pretrained = True
net = models.vgg16(pretrained=use_pretrained)
net.eval()
print(net)

/Users/phatvu/miniconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/phatvu/miniconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [4]:
class BaseTransform():
    def __init__(self, resize, mean, std):
        self.base_transform = transforms.Compose([
            transforms.Resize(resize),
            transforms.CenterCrop(resize),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])
    
    def __call__(self, img):
        return self.base_transform(img)

In [ ]:
# !mkdir -p data

# import urllib.request
# url = "https://cdn.outsideonline.com/wp-content/uploads/2023/03/Funny_Dog_S.jpg"
# save_path = "./data/dog.jpg"
# urllib.request.urlretrieve(url, save_path)

('./data/dog.jpg', <http.client.HTTPMessage at 0x12fac14d0>)

In [35]:
# ImageNet large scale visual recognition challenge (ILSVRC)
class ILSVRCPredictor():
    def __init__(self, class_index):
        self.class_index = class_index
    
    def predict_max(self, out):
        max_id = np.argmax(out.detach().numpy())
        predicted_label_name = self.class_index[str(max_id)]
        return predicted_label_name

In [10]:
# !curl -O https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json
# !mv imagenet_class_index.json ./data/imagenet_class_index.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 35363  100 35363    0     0  29617      0  0:00:01  0:00:01 --:--:-- 29642


In [12]:
class_index = json.load(open("./data/imagenet_class_index.json", "r"))

In [36]:
predictor = ILSVRCPredictor(class_index)

In [22]:
img_dog_path = "./data/dog.jpg"
img = Image.open(img_dog_path)

resize = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

transform = BaseTransform(resize, mean, std)
img_transformed = transform(img)
img_transformed = img_transformed.unsqueeze_(0)
print(img_transformed.size())

torch.Size([1, 3, 224, 224])


In [37]:
out = net(img_transformed)
result = predictor.predict_max(out)
print("Predicted Result: ", result)

Predicted Result:  ['n02085620', 'Chihuahua']
